# Red Neuronal

### Imports

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from joblib import dump, load

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score

In [ ]:
from preprocessing import reemplazarNulls,reemplazarCategoricas,reemplazarFechas,targetBooleano,regularizar

### Codigo a correr

In [ ]:
MAX_ITER = 10**4
MAX_JOBS = 1

**Run :** Entrenar Nuevamente o Cargar Entrenado

In [ ]:
runSimple = False
runMlpC = False
runMlpC_R = False

**Save :** Guardar Modelo (pisa anterior)

In [ ]:
saveMlpC = True
saveMlpC_R = True

In [ ]:
from utilities import score, df_feat, df_targ, train_val

### Preprocesamiento

In [ ]:
targetBooleano(df_targ, inplace=True)
reemplazarNulls(df_feat , inplace=True)
reemplazarCategoricas(df_feat , inplace=True)
reemplazarFechas(df_feat , inplace=True)
df_feat_r = regularizar(df_feat)

In [ ]:
train_feat, val_feat, train_targ, val_targ = train_val( df_feat, df_targ )
train_targ = train_targ.llovieron_hamburguesas_al_dia_siguiente
val_targ = val_targ.llovieron_hamburguesas_al_dia_siguiente

In [ ]:
train_feat_r, val_feat_r, train_targ_r, val_targ_r = train_val( df_feat_r, df_targ )
train_targ_r = train_targ_r.llovieron_hamburguesas_al_dia_siguiente
val_targ_r = val_targ_r.llovieron_hamburguesas_al_dia_siguiente

In [ ]:
layer_sizes = []
for i in range(10):
    for j in range(10):
        layer_sizes.append( ( i+1, j+1 ) )

## Simple

In [ ]:
if (runSimple):
    mlp = MLPClassifier(
        solver='adam',
        max_iter=MAX_ITER,
        random_state=1,
    ).fit(train_feat, train_targ)

In [ ]:
if (runSimple):
    pred = mlp.predict(val_feat)
    prob = mlp.predict_proba(val_feat)
    display(score(val_targ, pred, prob[:,1]))

## MLP Clasifier

In [ ]:
if (runMlpC):
    mlp_cls = MLPClassifier(
        solver='adam',
        max_iter=MAX_ITER,
        random_state=1,
    )
    params = {
        'alpha': np.arange(1e-5,10e-5,1e-5),
        'hidden_layer_sizes':layer_sizes,
        'activation': ['identity', 'logistic', 'relu', 'tanh'],
        'learning_rate': ["constant", "invscaling", "adaptive"],
    }
    
    mlp_cls_grid = RandomizedSearchCV(
        mlp_cls,
        params,
        scoring='roc_auc',
        n_jobs=MAX_JOBS,
        return_train_score=True,
        verbose=1,
    ).fit(train_feat, train_targ)
    
    # Guardar
    if(saveMlpC):
        dump(mlp_cls_grid, 'models/NN/mlp_cls_grid.sk')
        dump(mlp_cls_grid.best_estimator_, 'models/NN/mlp_cls.sk')
else:
    # Cargar
    mlp_cls_grid = load('models/NN/mlp_cls_grid.sk')

In [ ]:
print(f"Best score: {mlp_cls_grid.best_score_}")
print(f"Best params {mlp_cls_grid.best_params_}")

In [ ]:
mlp_cls = mlp_cls_grid.best_estimator_

In [ ]:
%%time
mlp_pred = mlp_cls.predict(val_feat)
mlp_prob = mlp_cls.predict_proba(val_feat)
score(val_targ, mlp_pred, mlp_prob[:,1])

## MPL Clasifier - Features Regularizados

In [ ]:
if (runMlpC_R):
    mlp_cls_r = MLPClassifier(
        solver='adam',
        max_iter=MAX_ITER,
        random_state=1,
    )
    params = {
        'alpha': np.arange(1e-5,10e-5,1e-5),
        'hidden_layer_sizes':layer_sizes,
        'activation': ['identity', 'logistic', 'relu', 'tanh'],
        'learning_rate': ["constant", "invscaling", "adaptive"],
    }
    
    mlp_cls_r_grid = RandomizedSearchCV(
        mlp_cls_r,
        params,
        scoring='roc_auc',
        n_jobs=MAX_JOBS,
        return_train_score=True,
        verbose=1,
    ).fit(train_feat_r, train_targ_r)
    
    # Guardar
    if(saveMlpC_R):
        dump(mlp_cls_r_grid, 'models/NN/mlp_cls_r_grid.sk')
        dump(mlp_cls_r_grid.best_estimator_, 'models/NN/mlp_cls_r.sk')
else:
    # Cargar
    mlp_cls_r_grid = load('models/NN/mlp_cls_r_grid.sk')

In [ ]:
print(f"Best score: {mlp_cls_r_grid.best_score_}")
print(f"Best params {mlp_cls_r_grid.best_params_}")

In [ ]:
mlp_cls_r = mlp_cls_r_grid.best_estimator_

In [ ]:
%%time
mlp_r_pred = mlp_cls_r.predict(val_feat_r)
mlp_r_prob = mlp_cls_r.predict_proba(val_feat_r)
score(val_targ_r, mlp_r_pred, mlp_r_prob[:,1])